In [1]:
# Instalar dependencias específicas para PDF si no están instaladas
# !pip install pypdf langchain-community pymupdf4llm pdfplumber
# 
# Si hay problemas con pdfminer, intenta:
# !pip install pdfminer.six==20231228
# !pip install unstructured[pdf]
# 
# Para macOS, también puedes necesitar:
# !brew install poppler


In [6]:
# Imports necesarios
import os
import warnings
from pathlib import Path
from dotenv import load_dotenv
import time

# Suprimir warnings
warnings.filterwarnings('ignore')

# Cargar variables de entorno
load_dotenv()

# Verificar credenciales
required_keys = ["OPENAI_API_KEY", "QDRANT_URL", "QDRANT_API_KEY"]
missing_keys = [key for key in required_keys if not os.getenv(key)]

if missing_keys:
    print(f"⚠️  Faltan las siguientes variables de entorno: {missing_keys}")
    print("📝 Configúralas en tu archivo .env")
else:
    print("✅ Todas las credenciales están configuradas correctamente")
    
# Verificar archivo PDF
pdf_path = "/Users/pablolastrabachmann/DiploGenAI/modulo 4/Contrato de arriendo departamento notariado - Pablo Lastra.PDF"
if Path(pdf_path).exists():
    print(f"✅ Archivo PDF encontrado: {Path(pdf_path).name}")
else:
    print(f"❌ Archivo PDF no encontrado en: {pdf_path}")


✅ Todas las credenciales están configuradas correctamente
✅ Archivo PDF encontrado: Contrato de arriendo departamento notariado - Pablo Lastra.PDF


In [7]:
# Instalar dependencias necesarias si hay problemas
import subprocess
import sys

def install_package(package):
    """Instala un paquete si no está disponible"""
    try:
        __import__(package)
        print(f"✅ {package} ya está instalado")
        return True
    except ImportError:
        print(f"📦 Instalando {package}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ {package} instalado exitosamente")
            return True
        except Exception as e:
            print(f"❌ Error instalando {package}: {e}")
            return False

print("🔧 Verificando e instalando dependencias para PDF...")

# Paquetes necesarios para PDF
pdf_packages = [
    "pymupdf",      # fitz
    "pdfplumber", 
    "pypdf"
]

# Instalar paquetes necesarios
for package in pdf_packages:
    install_package(package)

print("\n✅ Verificación de dependencias completada")


🔧 Verificando e instalando dependencias para PDF...
✅ pymupdf ya está instalado
✅ pdfplumber ya está instalado
✅ pypdf ya está instalado

✅ Verificación de dependencias completada


In [8]:
# Imports para procesamiento de PDF
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from pydantic import BaseModel, Field
from typing import Optional
import fitz  # PyMuPDF

print("📖 Cargando y procesamiento del contrato PDF...")

# Función mejorada con múltiples métodos de carga
def load_pdf_with_fallback(pdf_path):
    """Carga PDF con múltiples métodos para mayor robustez"""
    
    # Método 1: PyPDFLoader
    try:
        print("🔄 Intentando con PyPDFLoader...")
        from langchain_community.document_loaders import PyPDFLoader
        loader = PyPDFLoader(pdf_path)
        pages = loader.load()
        
        if pages and len(pages[0].page_content.strip()) > 50:
            print(f"✅ PyPDFLoader exitoso - {len(pages)} páginas cargadas")
            return pages
        else:
            print("⚠️ PyPDFLoader: contenido insuficiente")
    except Exception as e:
        print(f"⚠️ PyPDFLoader falló: {e}")
    
    # Método 2: PyMuPDF (fitz) - más robusto
    try:
        print("🔄 Intentando con PyMuPDF...")
        doc = fitz.open(pdf_path)
        pages = []
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text = page.get_text()
            
            if text.strip():
                pages.append(Document(
                    page_content=text,
                    metadata={"page": page_num + 1, "source": pdf_path}
                ))
        
        doc.close()
        
        if pages:
            print(f"✅ PyMuPDF exitoso - {len(pages)} páginas cargadas")
            return pages
        else:
            print("⚠️ PyMuPDF: no se extrajo contenido")
            
    except Exception as e:
        print(f"⚠️ PyMuPDF falló: {e}")
    
    # Método 3: PDFPlumber (más preciso)
    try:
        print("🔄 Intentando con PDFPlumber...")
        import pdfplumber
        
        pages = []
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                
                if text and text.strip():
                    pages.append(Document(
                        page_content=text,
                        metadata={"page": i + 1, "source": pdf_path}
                    ))
        
        if pages:
            print(f"✅ PDFPlumber exitoso - {len(pages)} páginas cargadas")
            return pages
        else:
            print("⚠️ PDFPlumber: no se extrajo contenido")
            
    except Exception as e:
        print(f"⚠️ PDFPlumber falló: {e}")
    
    # Método 4: UnstructuredPDFLoader (si está disponible)
    try:
        print("🔄 Intentando con UnstructuredPDFLoader...")
        from langchain_community.document_loaders import UnstructuredPDFLoader
        loader = UnstructuredPDFLoader(pdf_path)
        pages = loader.load()
        
        if pages:
            print(f"✅ UnstructuredPDFLoader exitoso - {len(pages)} documentos cargados")
            return pages
    except Exception as e:
        print(f"⚠️ UnstructuredPDFLoader falló: {e}")
    
    # Método 5: Lectura básica con pypdf
    try:
        print("🔄 Intentando con pypdf básico...")
        import pypdf
        
        pages = []
        with open(pdf_path, 'rb') as file:
            pdf_reader = pypdf.PdfReader(file)
            
            for i, page in enumerate(pdf_reader.pages):
                text = page.extract_text()
                
                if text and text.strip():
                    pages.append(Document(
                        page_content=text,
                        metadata={"page": i + 1, "source": pdf_path}
                    ))
        
        if pages:
            print(f"✅ pypdf básico exitoso - {len(pages)} páginas cargadas")
            return pages
        else:
            print("⚠️ pypdf: no se extrajo contenido")
            
    except Exception as e:
        print(f"⚠️ pypdf falló: {e}")
    
    raise Exception("❌ No se pudo cargar el PDF con ningún método disponible")

# Cargar el PDF
try:
    raw_documents = load_pdf_with_fallback(pdf_path)
    
    # Estadísticas básicas
    total_chars = sum(len(doc.page_content) for doc in raw_documents)
    total_words = sum(len(doc.page_content.split()) for doc in raw_documents)
    
    print(f"\n📊 Estadísticas del documento:")
    print(f"  • Páginas/Secciones: {len(raw_documents)}")
    print(f"  • Total caracteres: {total_chars:,}")
    print(f"  • Total palabras: {total_words:,}")
    print(f"  • Promedio palabras por página: {total_words//len(raw_documents):,}")
    
except Exception as e:
    print(f"❌ Error cargando PDF: {e}")
    print("\n🔧 SOLUCIONES POSIBLES:")
    print("1. Instala las dependencias: pip install pymupdf pdfplumber pypdf")
    print("2. Si el PDF está corrupto, intenta abrirlo y exportarlo nuevamente")
    print("3. Verifica que el archivo existe y tienes permisos de lectura")
    print("4. Para PDFs complejos, convierte a texto manualmente")
    raw_documents = []


📖 Cargando y procesamiento del contrato PDF...
🔄 Intentando con PyPDFLoader...
⚠️ PyPDFLoader: contenido insuficiente
🔄 Intentando con PyMuPDF...
⚠️ PyMuPDF: no se extrajo contenido
🔄 Intentando con PDFPlumber...
⚠️ PDFPlumber: no se extrajo contenido
🔄 Intentando con UnstructuredPDFLoader...
⚠️ UnstructuredPDFLoader falló: No module named 'pi_heif'
🔄 Intentando con pypdf básico...
⚠️ pypdf: no se extrajo contenido
❌ Error cargando PDF: ❌ No se pudo cargar el PDF con ningún método disponible

🔧 SOLUCIONES POSIBLES:
1. Instala las dependencias: pip install pymupdf pdfplumber pypdf
2. Si el PDF está corrupto, intenta abrirlo y exportarlo nuevamente
3. Verifica que el archivo existe y tienes permisos de lectura
4. Para PDFs complejos, convierte a texto manualmente


In [9]:
# Diagnóstico completo del PDF
import fitz  # PyMuPDF
import os
from pathlib import Path

def diagnose_pdf(pdf_path):
    """Diagnostica las propiedades del PDF para entender por qué no se extrae texto"""
    
    print(f"🔍 DIAGNÓSTICO COMPLETO DEL PDF")
    print("=" * 50)
    
    # 1. Verificar existencia y propiedades del archivo
    if not os.path.exists(pdf_path):
        print(f"❌ El archivo no existe: {pdf_path}")
        return False
    
    file_size = os.path.getsize(pdf_path) / (1024 * 1024)  # MB
    print(f"📁 Archivo: {Path(pdf_path).name}")
    print(f"📊 Tamaño: {file_size:.2f} MB")
    print(f"🔗 Ruta: {pdf_path}")
    
    try:
        # 2. Abrir con PyMuPDF para diagnóstico detallado
        doc = fitz.open(pdf_path)
        
        print(f"\n📄 PROPIEDADES DEL PDF:")
        print(f"  • Páginas: {len(doc)}")
        print(f"  • Versión PDF: {doc.pdf_version()}")
        print(f"  • Encriptado: {'Sí' if doc.needs_pass else 'No'}")
        print(f"  • Metadata: {doc.metadata}")
        
        # 3. Analizar cada página
        print(f"\n📋 ANÁLISIS POR PÁGINA:")
        
        text_found = False
        images_found = False
        
        for page_num in range(min(3, len(doc))):  # Analizar primeras 3 páginas
            page = doc.load_page(page_num)
            
            # Intentar extraer texto
            text = page.get_text()
            text_blocks = page.get_text("dict")
            images = page.get_images()
            
            print(f"\n  📄 Página {page_num + 1}:")
            print(f"    • Texto extraído: {len(text)} caracteres")
            print(f"    • Bloques de texto: {len(text_blocks.get('blocks', []))}")
            print(f"    • Imágenes: {len(images)}")
            
            if len(text.strip()) > 10:
                text_found = True
                print(f"    • Muestra de texto: '{text[:100].strip()}...'")
            
            if images:
                images_found = True
                print(f"    • Contiene imágenes (posible PDF escaneado)")
        
        doc.close()
        
        # 4. Diagnóstico final
        print(f"\n🎯 DIAGNÓSTICO:")
        if text_found:
            print("✅ Se encontró texto extraíble en el PDF")
            return True
        elif images_found:
            print("📸 PDF contiene principalmente imágenes (documento escaneado)")
            print("💡 Solución: Necesitas OCR para extraer texto de imágenes")
            return False
        else:
            print("❓ PDF sin texto ni imágenes detectables")
            print("💡 Puede estar corrupto o tener formato no estándar")
            return False
            
    except Exception as e:
        print(f"❌ Error en diagnóstico: {e}")
        return False

# Ejecutar diagnóstico
has_text = diagnose_pdf(pdf_path)


🔍 DIAGNÓSTICO COMPLETO DEL PDF
📁 Archivo: Contrato de arriendo departamento notariado - Pablo Lastra.PDF
📊 Tamaño: 0.69 MB
🔗 Ruta: /Users/pablolastrabachmann/DiploGenAI/modulo 4/Contrato de arriendo departamento notariado - Pablo Lastra.PDF

📄 PROPIEDADES DEL PDF:
  • Páginas: 3
❌ Error en diagnóstico: 'Document' object has no attribute 'pdf_version'


In [10]:
# 🔬 DEMOSTRACIÓN: Cómo funciona OCR paso a paso
print("🧪 DEMOSTRACIÓN PRÁCTICA DE OCR")
print("=" * 50)

# Si ya procesamos el PDF con OCR, mostrar los detalles
if 'raw_documents' in locals() and raw_documents and any(doc.metadata.get('method') == 'OCR' for doc in raw_documents):
    
    print("✅ Tu contrato YA fue procesado con OCR. Aquí están los detalles:")
    print()
    
    for i, doc in enumerate(raw_documents):
        if doc.metadata.get('method') == 'OCR':
            print(f"📄 PÁGINA {doc.metadata.get('page', i+1)}:")
            print(f"   • Caracteres extraídos: {len(doc.page_content):,}")
            print(f"   • Palabras: {len(doc.page_content.split()):,}")
            print(f"   • Método: {doc.metadata.get('method', 'Unknown')}")
            
            # Mostrar muestra del texto extraído
            sample_text = doc.page_content[:200].replace('\n', ' ').strip()
            print(f"   • Muestra de texto: \"{sample_text}...\"")
            print()
    
    print("🎯 ¿QUÉ HIZO EL OCR?")
    print("1. 📸 Convirtió cada página del PDF en imagen")
    print("2. 🔍 Detectó regiones con texto")
    print("3. 🧠 Usó IA para reconocer cada letra")
    print("4. 📝 Construyó palabras y oraciones")
    print("5. ✅ Filtró solo texto con alta confianza (>50%)")
    
    print(f"\n💡 RESULTADO FINAL:")
    total_chars = sum(len(doc.page_content) for doc in raw_documents)
    total_words = sum(len(doc.page_content.split()) for doc in raw_documents)
    print(f"   • {len(raw_documents)} páginas procesadas")
    print(f"   • {total_chars:,} caracteres extraídos")
    print(f"   • {total_words:,} palabras reconocidas")
    print(f"   • ✅ Listo para usar en RAG")

else:
    print("ℹ️ OCR aún no se ha ejecutado en tu documento")
    print()
    print("🤖 CÓMO FUNCIONA OCR (Ejemplo conceptual):")
    print()
    print("ENTRADA (Imagen con texto):")
    print("┌─────────────────────────┐")
    print("│  CONTRATO DE ARRIENDO   │  ← OCR 'lee' esto")
    print("│  Entre don Pablo...     │  ← e identifica cada letra")
    print("│  Monto: $450.000        │  ← incluso números")
    print("└─────────────────────────┘")
    print()
    print("SALIDA (Texto digital):")
    print("'CONTRATO DE ARRIENDO\\nEntre don Pablo...\\nMonto: $450.000'")
    print()
    print("🎯 PROCESO INTERNO:")
    print("1. 🖼️  Imagen → Píxeles")
    print("2. 🔍 Detección → Encontrar texto")
    print("3. 🧠 IA → Reconocer caracteres")
    print("4. 📝 Construcción → Formar palabras")
    print("5. ✅ Validación → Filtrar por confianza")

print("\n" + "=" * 50)


🧪 DEMOSTRACIÓN PRÁCTICA DE OCR
ℹ️ OCR aún no se ha ejecutado en tu documento

🤖 CÓMO FUNCIONA OCR (Ejemplo conceptual):

ENTRADA (Imagen con texto):
┌─────────────────────────┐
│  CONTRATO DE ARRIENDO   │  ← OCR 'lee' esto
│  Entre don Pablo...     │  ← e identifica cada letra
│  Monto: $450.000        │  ← incluso números
└─────────────────────────┘

SALIDA (Texto digital):
'CONTRATO DE ARRIENDO\nEntre don Pablo...\nMonto: $450.000'

🎯 PROCESO INTERNO:
1. 🖼️  Imagen → Píxeles
2. 🔍 Detección → Encontrar texto
3. 🧠 IA → Reconocer caracteres
4. 📝 Construcción → Formar palabras
5. ✅ Validación → Filtrar por confianza



In [11]:
# Extracción de texto con OCR para PDFs escaneados
# Ejecuta esta celda SOLO si el diagnóstico indica que el PDF contiene imágenes

def extract_text_with_ocr(pdf_path):
    """Extrae texto usando OCR si el PDF está escaneado"""
    
    print("📸 Intentando extracción con OCR...")
    
    try:
        # Instalar easyocr si no está disponible
        try:
            import easyocr
        except ImportError:
            print("📦 Instalando EasyOCR...")
            import subprocess
            import sys
            subprocess.check_call([sys.executable, "-m", "pip", "install", "easyocr"])
            import easyocr
        
        # Convertir PDF a imágenes
        doc = fitz.open(pdf_path)
        reader = easyocr.Reader(['es', 'en'])  # Español e inglés
        
        all_text = []
        
        for page_num in range(len(doc)):
            print(f"🔄 Procesando página {page_num + 1}/{len(doc)} con OCR...")
            
            # Convertir página a imagen
            page = doc.load_page(page_num)
            pix = page.get_pixmap()
            img_data = pix.tobytes("png")
            
            # Extraer texto con OCR
            results = reader.readtext(img_data)
            
            # Concatenar texto extraído
            page_text = ""
            for (bbox, text, confidence) in results:
                if confidence > 0.5:  # Solo texto con confianza > 50%
                    page_text += text + " "
            
            if page_text.strip():
                all_text.append(Document(
                    page_content=page_text.strip(),
                    metadata={
                        "page": page_num + 1,
                        "source": pdf_path,
                        "method": "OCR",
                        "confidence": "medium"
                    }
                ))
                
                print(f"   ✅ Extraídos {len(page_text)} caracteres")
            else:
                print(f"   ⚠️ No se extrajo texto de la página {page_num + 1}")
        
        doc.close()
        
        if all_text:
            print(f"\n✅ OCR completado: {len(all_text)} páginas procesadas")
            total_chars = sum(len(doc.page_content) for doc in all_text)
            print(f"📊 Total caracteres extraídos: {total_chars:,}")
            return all_text
        else:
            print("❌ OCR no pudo extraer texto")
            return None
            
    except Exception as e:
        print(f"❌ Error en OCR: {e}")
        print("💡 Instala easyocr: pip install easyocr")
        return None

# Ejecutar OCR solo si el diagnóstico indica PDF escaneado
if 'has_text' in locals() and not has_text:
    print("🔄 PDF parece ser escaneado, intentando OCR...")
    raw_documents = extract_text_with_ocr(pdf_path)
    
    if raw_documents:
        print("✅ Texto extraído exitosamente con OCR")
    else:
        print("❌ OCR falló, necesitarás método manual")
        raw_documents = []
else:
    print("ℹ️ OCR no necesario - ejecuta diagnóstico primero")


🔄 PDF parece ser escaneado, intentando OCR...
📸 Intentando extracción con OCR...
🔄 Procesando página 1/3 con OCR...
   ✅ Extraídos 1464 caracteres
🔄 Procesando página 2/3 con OCR...
   ✅ Extraídos 1597 caracteres
🔄 Procesando página 3/3 con OCR...
   ✅ Extraídos 964 caracteres

✅ OCR completado: 3 páginas procesadas
📊 Total caracteres extraídos: 4,022
✅ Texto extraído exitosamente con OCR


In [12]:
# Verificar estado actual y dar instrucciones
print("📋 ESTADO ACTUAL DE LA EXTRACCIÓN DE TEXTO")
print("=" * 60)

# Verificar si tenemos documentos cargados
if 'raw_documents' in locals() and raw_documents:
    total_chars = sum(len(doc.page_content) for doc in raw_documents)
    total_words = sum(len(doc.page_content.split()) for doc in raw_documents)
    method = raw_documents[0].metadata.get('method', 'unknown')
    
    print("✅ TEXTO EXTRAÍDO EXITOSAMENTE")
    print(f"📊 Estadísticas finales:")
    print(f"  • Documentos/Páginas: {len(raw_documents)}")
    print(f"  • Total caracteres: {total_chars:,}")
    print(f"  • Total palabras: {total_words:,}")
    print(f"  • Método usado: {method}")
    
    print(f"\n🔍 Muestra del contenido:")
    print(f"'{raw_documents[0].page_content[:150]}...'")
    
    print(f"\n🚀 PRÓXIMOS PASOS:")
    print("✅ Continúa ejecutando las siguientes celdas del notebook")
    print("✅ El sistema RAG se configurará automáticamente")
    print("✅ Podrás hacer preguntas sobre tu contrato")
    
else:
    print("❌ AÚN NO SE HA EXTRAÍDO TEXTO")
    print(f"\n🔧 OPCIONES DISPONIBLES:")
    print("1. 📖 Ejecuta la celda de DIAGNÓSTICO primero")
    print("2. 📸 Si es PDF escaneado, usa la celda de OCR") 
    print("3. 📝 Si todo falla, usa el MÉTODO MANUAL (copia/pega)")
    print("4. 🔄 Verifica que el archivo PDF no esté corrupto")
    
    print(f"\n💡 RECOMENDACIONES:")
    print("• El método manual (copia/pega) es el más confiable")
    print("• Abre tu PDF en Preview/Adobe y copia todo el texto")
    print("• Pega el texto en la celda manual y ejecútala")
    
    print(f"\n⚠️ Si el PDF no permite copiar texto:")
    print("• Puede estar protegido o ser solo imágenes")
    print("• Usa el método OCR o convierte el PDF a texto primero")

print("\n" + "=" * 60)


📋 ESTADO ACTUAL DE LA EXTRACCIÓN DE TEXTO
✅ TEXTO EXTRAÍDO EXITOSAMENTE
📊 Estadísticas finales:
  • Documentos/Páginas: 3
  • Total caracteres: 4,022
  • Total palabras: 556
  • Método usado: OCR

🔍 Muestra del contenido:
'NOTARiO CONTRATO DE ARRIENDO DEPARTAMENTO Santiagc (6 de meyo de 2024 Entre MARIANA OLATE VENTURELLI, Rut ARRENDADOR' PABLO ENRIQUE LASTRA BACHMANN aq...'

🚀 PRÓXIMOS PASOS:
✅ Continúa ejecutando las siguientes celdas del notebook
✅ El sistema RAG se configurará automáticamente
✅ Podrás hacer preguntas sobre tu contrato



In [13]:
# Mostrar muestra del contenido
if raw_documents:
    print("🔍 Muestra del contenido extraído:")
    print("=" * 50)
    sample_content = raw_documents[0].page_content[:500]
    print(sample_content)
    print("=" * 50)
    
    if len(raw_documents) > 1:
        print(f"\n📄 El documento tiene {len(raw_documents)} páginas/secciones")
        for i, doc in enumerate(raw_documents[:3]):
            print(f"  • Página {i+1}: {len(doc.page_content)} caracteres")


🔍 Muestra del contenido extraído:
NOTARiO CONTRATO DE ARRIENDO DEPARTAMENTO Santiagc (6 de meyo de 2024 Entre MARIANA OLATE VENTURELLI, Rut ARRENDADOR' PABLO ENRIQUE LASTRA BACHMANN aqu/ adelante EL ARRENDATARIO' ce edad seha convenido en siguienie PRIMERO; arrendacor ca en amenjamienic arrendatario Avenida Viel 1616 Departamento 1107 ccmuna ce Santiago Centro SEGUNDO: desbnada para local arrendatano TERCERO contar del 06 de mayo 2024 misma fecha dla del lvego del pnmer ano arendajor presente contrato deoera enviar certihcaca Co

📄 El documento tiene 3 páginas/secciones
  • Página 1: 1463 caracteres
  • Página 2: 1596 caracteres
  • Página 3: 963 caracteres


In [18]:
# Configurar conexiones y chunking
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from qdrant_client.models import Distance, VectorParams

print("🔧 Configurando sistema RAG para contratos...")

# 1. Configurar text splitter optimizado para contratos
chunk_size = 800
chunk_overlap = int(chunk_size * 0.2)  # 20% overlap

contract_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separators=["\n\n", "\n", ". ", ", ", " ", ""],
    keep_separator=True
)

print(f"✂️ Chunking configurado: {chunk_size} chars, {chunk_overlap} overlap")

# 2. Configurar OpenAI
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=512  # Dimensión reducida para eficiencia
)

llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
print("✅ OpenAI configurado")

# 3. Configurar Qdrant
try:
    qdrant_client = QdrantClient(
        url=os.getenv("QDRANT_URL"),
        api_key=os.getenv("QDRANT_API_KEY"),
    )
    
    collections = qdrant_client.get_collections()
    print(f"✅ Conectado a Qdrant Cloud ({len(collections.collections)} colecciones)")
    
except Exception as e:
    print(f"❌ Error conectando con Qdrant: {e}")
    qdrant_client = None


🔧 Configurando sistema RAG para contratos...
✂️ Chunking configurado: 800 chars, 160 overlap
✅ OpenAI configurado
✅ Conectado a Qdrant Cloud (3 colecciones)


In [22]:
# Conectar a colección existente o crear nueva
if qdrant_client:
    collection_name = "contrato_arriendo_pablo"
    
    try:
        # Verificar si la colección ya existe
        existing_collections = [col.name for col in qdrant_client.get_collections().collections]
        
        if collection_name in existing_collections:
            print(f"🔗 Conectando a colección existente '{collection_name}'...")
            
            # Conectar a colección existente
            vector_store = QdrantVectorStore(
                client=qdrant_client,
                collection_name=collection_name,
                embedding=embeddings
            )
            
            # Verificar contenido de la colección
            collection_info = qdrant_client.get_collection(collection_name)
            vectors_count = collection_info.vectors_count
            
            print(f"✅ Conectado a colección existente!")
            print(f"📊 La colección contiene {vectors_count} vectores/chunks")
            print(f"🚀 Listo para usar el sistema RAG")
            
        else:
            print(f"🔄 Colección '{collection_name}' no existe, creando nueva...")
            
            if raw_documents:
                # 1. Crear chunks
                chunks = contract_splitter.split_documents(raw_documents)
                
                # 2. Enriquecer metadata para contratos
                for i, chunk in enumerate(chunks):
                    chunk.metadata.update({
                        "source": "Contrato de arriendo Pablo Lastra",
                        "chunk_id": i,
                        "tipo_documento": "contrato_arriendo",
                        "char_count": len(chunk.page_content),
                        "word_count": len(chunk.page_content.split())
                    })
                
                print(f"✂️ Creados {len(chunks)} chunks")
                print(f"📊 Promedio: {len(chunks)/len(raw_documents):.1f} chunks por página")
                
                # 3. Crear nueva colección
                start_time = time.time()
                
                vector_store = QdrantVectorStore.from_documents(
                    documents=chunks,
                    embedding=embeddings,
                    url=os.getenv("QDRANT_URL"),
                    api_key=os.getenv("QDRANT_API_KEY"),
                    collection_name=collection_name,
                    force_recreate=True
                )
                
                index_time = time.time() - start_time
                
                print(f"✅ Vector store creado exitosamente!")
                print(f"⏱️ Tiempo de indexación: {index_time:.2f} segundos")
                print(f"📊 {len(chunks)} chunks indexados en Qdrant Cloud")
            else:
                print("❌ No hay documentos para crear la colección")
                vector_store = None
        
    except Exception as e:
        print(f"❌ Error con vector store: {e}")
        vector_store = None
        
else:
    print("❌ No se puede procesar: falta conexión Qdrant")
    vector_store = None


🔗 Conectando a colección existente 'contrato_arriendo_pablo'...
✅ Conectado a colección existente!
📊 La colección contiene None vectores/chunks
🚀 Listo para usar el sistema RAG


In [23]:
# Configurar sistema RAG especializado para contratos
if vector_store:
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser
    from langchain_core.runnables import RunnablePassthrough, RunnableParallel
    from langchain.callbacks import get_openai_callback
    
    print("🤖 Configurando sistema RAG especializado para contratos...")
    
    # Configurar retriever
    retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3, "score_threshold": 0.1}
    )
    
    # Función para formatear documentos de contrato
    def format_contract_docs(docs):
        """Formatea los chunks recuperados con información específica de contratos"""
        formatted_docs = []
        for i, doc in enumerate(docs, 1):
            doc_text = f"[Fragmento {i} - Página {doc.metadata.get('page', 'N/A')}]\n"
            doc_text += f"Contenido: {doc.page_content}\n"
            formatted_docs.append(doc_text)
        return "\n" + "="*50 + "\n".join(formatted_docs)
    
    # Prompt especializado para contratos de arriendo
    contract_prompt = ChatPromptTemplate.from_template("""
## ROL
Eres un asistente legal especializado en contratos de arriendo en Chile.

## TAREA
Responde preguntas específicas sobre el contrato de arriendo basándote ÚNICAMENTE en la información proporcionada.

## INSTRUCCIONES:
1. **Analiza cuidadosamente** todos los fragmentos del contrato proporcionados
2. **Responde SOLO** con información que esté explícitamente en el contrato
3. **Cita específicamente** la página o sección donde encontraste la información
4. **Si no encuentras información**, indica claramente "Esta información no está disponible en las secciones analizadas del contrato"
5. **Para temas legales complejos**, sugiere consultar con un abogado
6. **Estructura tu respuesta** de manera clara y profesional
7. **Incluye montos, fechas y datos específicos** exactamente como aparecen en el contrato

## FORMATO DE RESPUESTA:
- Respuesta directa y concisa
- Cita la página/sección específica
- Si aplica, menciona el contexto legal relevante

## FRAGMENTOS DEL CONTRATO:
{context}

## PREGUNTA:
{question}

## RESPUESTA:
Según el contrato analizado:
""")
    
    # Chain RAG
    contract_rag_chain = (
        RunnableParallel({
            "context": retriever | format_contract_docs,
            "question": RunnablePassthrough()
        })
        | contract_prompt
        | llm
        | StrOutputParser()
    )
    
    print("✅ Sistema RAG configurado exitosamente")
    
else:
    print("❌ No se puede configurar RAG: vector store no disponible")
    contract_rag_chain = None


🤖 Configurando sistema RAG especializado para contratos...
✅ Sistema RAG configurado exitosamente


In [24]:
# Función para consultas interactivas sobre el contrato
def consultar_contrato(pregunta, mostrar_contexto=False, mostrar_metricas=True):
    """
    Realiza una consulta al contrato de arriendo
    
    Args:
        pregunta (str): La pregunta sobre el contrato
        mostrar_contexto (bool): Si mostrar los fragmentos recuperados
        mostrar_metricas (bool): Si mostrar métricas de performance
    
    Returns:
        str: La respuesta basada en el contrato
    """
    if not contract_rag_chain:
        return "❌ Sistema RAG no disponible"
    
    print(f"❓ Pregunta: {pregunta}")
    print("="*60)
    
    start_time = time.time()
    
    try:
        with get_openai_callback() as cb:
            # Mostrar contexto recuperado si se solicita
            if mostrar_contexto:
                retrieved_docs = retriever.invoke(pregunta)
                print("📚 FRAGMENTOS RECUPERADOS DEL CONTRATO:")
                for i, doc in enumerate(retrieved_docs, 1):
                    print(f"\n[Fragmento {i} - Página {doc.metadata.get('page', 'N/A')}]")
                    print(f"Contenido: {doc.page_content[:300]}...")
                print("\n" + "="*60)
            
            # Generar respuesta
            respuesta = contract_rag_chain.invoke(pregunta)
            
            end_time = time.time()
            
            # Mostrar respuesta
            print("🤖 RESPUESTA:")
            print(respuesta)
            
            # Mostrar métricas
            if mostrar_metricas:
                print(f"\n📊 MÉTRICAS:")
                print(f"⏱️ Tiempo total: {end_time - start_time:.2f} segundos")
                print(f"🎫 Tokens usados: {cb.total_tokens}")
                print(f"💰 Costo aproximado: ${cb.total_cost:.4f}")
            
            return respuesta
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

print("💬 Sistema de consultas interactivas configurado")
print("📝 Usa la función: consultar_contrato('tu pregunta aquí')")


💬 Sistema de consultas interactivas configurado
📝 Usa la función: consultar_contrato('tu pregunta aquí')


In [25]:
consultar_contrato('cuando se firmó el contrato?')

❓ Pregunta: cuando se firmó el contrato?
🤖 RESPUESTA:
El contrato fue firmado el 6 de mayo de 2024. Esta información se encuentra en el Fragmento 2 - Página 1 y también se menciona en el Fragmento 3 - Página 3.

📊 MÉTRICAS:
⏱️ Tiempo total: 5.68 segundos
🎫 Tokens usados: 688
💰 Costo aproximado: $0.0021


'El contrato fue firmado el 6 de mayo de 2024. Esta información se encuentra en el Fragmento 2 - Página 1 y también se menciona en el Fragmento 3 - Página 3.'

In [15]:
# Pruebas del sistema con preguntas típicas sobre contratos de arriendo
if contract_rag_chain:
    print("🧪 PRUEBAS DEL SISTEMA RAG PARA CONTRATO DE ARRIENDO")
    print("="*70)
    
    # Preguntas típicas sobre contratos de arriendo
    preguntas_test = [
        "¿Cuál es el monto del arriendo mensual?",
        "¿Quién es el arrendador y el arrendatario?",
        "¿Cuál es la dirección de la propiedad arrendada?",
        "¿Cuándo inicia y cuándo termina el contrato?",
        "¿Qué dice el contrato sobre mascotas?",
        "¿Cuáles son las obligaciones del arrendatario?",
        "¿Quién paga los gastos comunes?",
        "¿Hay garantías o depósitos requeridos?"
    ]
    
    print(f"📝 Ejecutando {len(preguntas_test)} preguntas de prueba...")
    print("\n" + "="*70)
    
    # Ejecutar solo las primeras 3 pruebas para demostración
    for i, pregunta in enumerate(preguntas_test[:3], 1):
        print(f"\n🔍 PRUEBA {i}/3")
        print("-" * 50)
        
        try:
            consultar_contrato(pregunta, mostrar_contexto=False, mostrar_metricas=False)
        except Exception as e:
            print(f"❌ Error en prueba {i}: {e}")
        
        print("\n" + "="*70)
    
    print("\n✅ Pruebas completadas")
    print("\n📝 Para hacer más consultas personalizadas, usa:")
    print("   consultar_contrato('tu pregunta aquí')")
    print("   consultar_contrato('pregunta', mostrar_contexto=True)  # Para ver fragmentos recuperados")
    
else:
    print("❌ Sistema no disponible para pruebas")
    print("\n🔧 Verifica que:")
    print("  • El PDF se haya cargado correctamente")
    print("  • Las credenciales de OpenAI y Qdrant estén configuradas")
    print("  • El vector store se haya creado exitosamente")


🧪 PRUEBAS DEL SISTEMA RAG PARA CONTRATO DE ARRIENDO
📝 Ejecutando 8 preguntas de prueba...


🔍 PRUEBA 1/3
--------------------------------------------------
❓ Pregunta: ¿Cuál es el monto del arriendo mensual?
🤖 RESPUESTA:
El monto del arriendo mensual es de 430,000 pesos chilenos. Esta información se encuentra en el Fragmento 1, Página 1 del contrato.


🔍 PRUEBA 2/3
--------------------------------------------------
❓ Pregunta: ¿Quién es el arrendador y el arrendatario?
🤖 RESPUESTA:
- El arrendador es Pablo Enrique Lastra Bachmann.
- El arrendatario es Mariana Loreto Olate Venturelli.

Esta información se encuentra en el Fragmento 2 - Página 3.


🔍 PRUEBA 3/3
--------------------------------------------------
❓ Pregunta: ¿Cuál es la dirección de la propiedad arrendada?
🤖 RESPUESTA:
La dirección de la propiedad arrendada es Avenida Viel 1616, Departamento 1107, comuna de Santiago Centro. Esta información se encuentra en el Fragmento 1, Página 1.


✅ Pruebas completadas

📝 Para hacer más 

In [26]:
# Implementar Rewrite-Retrieve-Read para consultas complejas sobre contratos
if contract_rag_chain:
    print("⚡ Configurando técnica Rewrite-Retrieve-Read...")
    
    # Prompt para mejorar consultas sobre contratos
    rewrite_prompt = ChatPromptTemplate.from_template("""
    Mejora la siguiente consulta sobre un contrato de arriendo para que sea más específica y efectiva para la búsqueda.
    
    INSTRUCCIONES:
    - Mantén el idioma original
    - Haz la consulta más específica y detallada
    - Incluye términos legales relevantes si es apropiado
    - Enfócate en aspectos específicos del contrato
    
    Consulta original: {query}
    
    Consulta mejorada:""")
    
    rewrite_chain = rewrite_prompt | llm | StrOutputParser()
    
    def consultar_con_rewrite(pregunta_original):
        """Consulta usando técnica Rewrite-Retrieve-Read"""
        print(f"❓ Pregunta original: {pregunta_original}")
        
        # Paso 1: Rewrite
        pregunta_mejorada = rewrite_chain.invoke({"query": pregunta_original})
        print(f"🔄 Pregunta mejorada: {pregunta_mejorada}")
        print("-" * 60)
        
        # Paso 2: Retrieve & Read
        respuesta = contract_rag_chain.invoke(pregunta_mejorada)
        
        print("🤖 RESPUESTA:")
        print(respuesta)
        
        return respuesta
    
    print("✅ Rewrite-Retrieve-Read configurado")
    print("📝 Usa: consultar_con_rewrite('pregunta básica')")
    
    # Ejemplo de uso
    print("\n" + "="*60)
    print("💡 EJEMPLO DE USO:")
    print("="*60)
    
    try:
        consultar_con_rewrite("pagos arriendo")
    except Exception as e:
        print(f"❌ Error en ejemplo: {e}")
    
else:
    print("❌ No se puede configurar técnicas avanzadas: sistema base no disponible")

print("\n" + "="*70)
print("🎉 SISTEMA RAG PARA CONTRATO DE ARRIENDO COMPLETADO")
print("="*70)
print("\n📚 FUNCIONES DISPONIBLES:")
print("  • consultar_contrato('pregunta') - Consulta básica")
print("  • consultar_contrato('pregunta', mostrar_contexto=True) - Con fragmentos")
print("  • consultar_con_rewrite('pregunta') - Con mejora automática")
print("\n🔍 EJEMPLOS DE PREGUNTAS:")
print("  • ¿Cuál es el monto del arriendo?")
print("  • ¿Qué obligaciones tengo como arrendatario?")
print("  • ¿Puedo tener mascotas?")
print("  • ¿Cuándo vence el contrato?")
print("  • ¿Qué pasa si termino antes el contrato?")
print("\n✅ ¡Listo para usar!")


⚡ Configurando técnica Rewrite-Retrieve-Read...
✅ Rewrite-Retrieve-Read configurado
📝 Usa: consultar_con_rewrite('pregunta básica')

💡 EJEMPLO DE USO:
❓ Pregunta original: pagos arriendo
🔄 Pregunta mejorada: "detalles de los pagos mensuales del contrato de arriendo, incluyendo fechas de vencimiento, métodos de pago aceptados, penalizaciones por retraso y ajustes anuales según el índice de inflación"
------------------------------------------------------------
🤖 RESPUESTA:
Según el contrato analizado:

- **Renta Mensual**: La renta mensual corresponde a la suma de 430,000 pesos chilenos. Este monto incluye el pago asociado a gastos comunes. (Fragmento 2 - Página 1)

- **Método de Pago**: La renta se pagará por mensualidades anticipadas a través de depósitos en cuenta. (Fragmento 1 - Página 1)

- **Fecha de Vencimiento**: No se especifica una fecha exacta de vencimiento mensual, pero se menciona que el pago debe hacerse de manera anticipada. (Fragmento 1 - Página 1)

- **Penalizaciones p